In [ ]:
import os
import numpy as np
import tensorflow as tf
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from shutil import copyfile
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import layers, losses, Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Upload dataset

In [ ]:
benign =pd.read_csv('/kaggle/input/nbaiot-dataset/1.benign.csv')
mirai_ack =pd.read_csv('/kaggle/input/nbaiot-dataset/1.mirai.ack.csv')
mirai_scan = pd.read_csv('/kaggle/input/nbaiot-dataset/1.mirai.scan.csv')
mirai_syn = pd.read_csv('/kaggle/input/nbaiot-dataset/1.mirai.syn.csv')
mirai_udp = pd.read_csv('/kaggle/input/nbaiot-dataset/1.mirai.udp.csv')
mirai_udp_plain = pd.read_csv('/kaggle/input/nbaiot-dataset/1.mirai.udpplain.csv')
gafgyt_combo = pd.read_csv('/kaggle/input/nbaiot-dataset/1.gafgyt.combo.csv')
gafgyt_junk = pd.read_csv('/kaggle/input/nbaiot-dataset/1.gafgyt.junk.csv')
gafgyt_scan = pd.read_csv('/kaggle/input/nbaiot-dataset/1.gafgyt.scan.csv')
gafgyt_tcp = pd.read_csv('/kaggle/input/nbaiot-dataset/1.gafgyt.tcp.csv')
gafgyt_udp = pd.read_csv('/kaggle/input/nbaiot-dataset/1.gafgyt.udp.csv')


#    unseen dataset

In [ ]:
benign2 =pd.read_csv('/kaggle/input/nbaiot-dataset/2.benign.csv')
mirai_scan2 = pd.read_csv('/kaggle/input/nbaiot-dataset/2.mirai.scan.csv')
mirai_ack2 =pd.read_csv('/kaggle/input/nbaiot-dataset/2.mirai.ack.csv')
mirai_scan2 = pd.read_csv('/kaggle/input/nbaiot-dataset/2.mirai.scan.csv')
mirai_syn2 = pd.read_csv('/kaggle/input/nbaiot-dataset/2.mirai.syn.csv')
mirai_udp2 = pd.read_csv('/kaggle/input/nbaiot-dataset/2.mirai.udp.csv')
mirai_udp_plain2 = pd.read_csv('/kaggle/input/nbaiot-dataset/2.mirai.udpplain.csv')
gafgyt_combo2 = pd.read_csv('/kaggle/input/nbaiot-dataset/2.gafgyt.combo.csv')
gafgyt_junk2 = pd.read_csv('/kaggle/input/nbaiot-dataset/2.gafgyt.junk.csv')
gafgyt_scan2 = pd.read_csv('/kaggle/input/nbaiot-dataset/2.gafgyt.scan.csv')
gafgyt_tcp2 = pd.read_csv('/kaggle/input/nbaiot-dataset/2.gafgyt.tcp.csv')
gafgyt_udp2 = pd.read_csv('/kaggle/input/nbaiot-dataset/2.gafgyt.udp.csv')


In [ ]:
benign = benign.sample(frac=0.50, replace=False)
mirai_ack = mirai_ack.sample(frac=0.24, replace=False)
mirai_scan = mirai_scan.sample(frac=0.22, replace=False)
mirai_syn = mirai_syn.sample(frac=0.2, replace=False)
mirai_udp = mirai_udp.sample(frac=0.1, replace=False)
mirai_udp_plain = mirai_udp_plain.sample(frac=0.3, replace=False)
gafgyt_combo = gafgyt_combo.sample(frac=0.4, replace=False)
gafgyt_junk = gafgyt_junk.sample(frac=0.8, replace=False)
gafgyt_scan = gafgyt_scan.sample(frac=0.8, replace=False)
gafgyt_tcp = gafgyt_tcp.sample(frac=0.25, replace=False)
gafgyt_udp = gafgyt_udp.sample(frac=0.23, replace=False)

benign['type']='benign'
mirai_ack['type']='mirai_ack'
mirai_scan['type']='mirai_scan'
mirai_syn['type'] = 'mirai_syn'
mirai_udp['type'] = 'mirai_udp'
mirai_udp_plain['type'] = 'mirai_udp_plain'

gafgyt_combo['type'] = 'gafgyt_combo'
gafgyt_junk['type'] = 'gafgyt_junk'
gafgyt_scan['type'] = 'gafgyt_scan'
gafgyt_tcp['type'] = 'gafgyt_tcp'
gafgyt_udp['type'] = 'gafgyt_udp'

data = pd.concat([benign, 
                  mirai_ack, mirai_scan, mirai_syn, mirai_udp, mirai_udp_plain,
                 gafgyt_combo, gafgyt_junk, gafgyt_scan, gafgyt_tcp, gafgyt_udp], 
                 axis=0, sort=False, ignore_index=True)


In [ ]:
benign2 = benign2.sample(frac=0.50, replace=False)
mirai_ack2 = mirai_ack2.sample(frac=0.24, replace=False)
mirai_scan2 = mirai_scan2.sample(frac=0.22, replace=False)

mirai_syn2 = mirai_syn2.sample(frac=0.2, replace=False)
mirai_udp2 = mirai_udp2.sample(frac=0.1, replace=False)
mirai_udp_plain2 = mirai_udp_plain2.sample(frac=0.3, replace=False)

gafgyt_combo2 = gafgyt_combo2.sample(frac=0.4, replace=False)
gafgyt_junk2 = gafgyt_junk2.sample(frac=0.8, replace=False)
gafgyt_scan2 = gafgyt_scan2.sample(frac=0.8, replace=False)
gafgyt_tcp2 = gafgyt_tcp2.sample(frac=0.25, replace=False)
gafgyt_udp2 = gafgyt_udp2.sample(frac=0.23, replace=False)

benign2['type']='benign'
mirai_ack2['type']='mirai_ack'
mirai_scan2['type']='mirai_scan'

mirai_syn2['type'] = 'mirai_syn'
mirai_udp2['type'] = 'mirai_udp'
mirai_udp_plain2['type'] = 'mirai_udp_plain'

gafgyt_combo2['type'] = 'gafgyt_combo'
gafgyt_junk2['type'] = 'gafgyt_junk'
gafgyt_scan2['type'] = 'gafgyt_scan'
gafgyt_tcp2['type'] = 'gafgyt_tcp'
gafgyt_udp2['type'] = 'gafgyt_udp'

unseen_data = pd.concat([benign2, 
                  mirai_ack2, mirai_scan2, mirai_syn2, mirai_udp2, mirai_udp_plain2,
                 gafgyt_combo2, gafgyt_junk2, gafgyt_scan2, gafgyt_tcp2, gafgyt_udp2], 
                 axis=0, sort=False, ignore_index=True)


In [ ]:
#unseen_data = pd.concat([benign2, scan],axis=0, sort=False, ignore_index=True)

In [ ]:
data=data.iloc[:, :28].join(data['type'])
unseen_data=unseen_data.iloc[:, :28].join(unseen_data['type'])

In [ ]:
data.groupby('type')['type'].count()

In [ ]:
unseen_data.groupby('type')['type'].count()

In [ ]:
benign.describe()

In [ ]:
benign2.describe()

In [ ]:
#Shuffling rows of the dataframe
sampler = np.random.permutation(len(data))
data = data.take(sampler)
data.head()

In [ ]:
#Shuffling rows of the dataframe
sampler2 = np.random.permutation(len(unseen_data))
unseen_data = data.take(sampler2)
unseen_data.head()

In [ ]:
features = data.drop(['type'], axis=1)
target = data.filter(['type'])

In [ ]:
features2 = unseen_data.drop(['type'], axis=1)
target2 = unseen_data.filter(['type'])

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data_st = scaler.fit_transform(features)

In [ ]:
unseen_data_st = scaler.fit_transform(features2)

In [ ]:
# Encode target column data
target.type = target.type.replace('benign', 0).replace('mirai_ack', 1).replace('mirai_scan', 2).replace('mirai_syn', 3).replace('mirai_udp', 4).replace('mirai_udp_plain', 5).replace('gafgyt_combo', 6).replace('gafgyt_junk', 7).replace('gafgyt_scan', 8).replace('gafgyt_tcp', 9).replace('gafgyt_udp', 10)

In [ ]:
target2.type = target2.type.replace('benign', 0).replace('mirai_ack', 1).replace('mirai_scan', 2).replace('mirai_syn', 3).replace('mirai_udp', 4).replace('mirai_udp_plain', 5).replace('gafgyt_combo', 6).replace('gafgyt_junk', 7).replace('gafgyt_scan', 8).replace('gafgyt_tcp', 9).replace('gafgyt_udp', 10)

In [ ]:
# Create dataset
data_v2 = {'data': data_st, 'target': np.array(target['type'].values)} 

In [ ]:
# Create dataset
unseen_data_v2 = {'u_data': unseen_data_st, 'u_target': np.array(target2['type'].values)} 

In [ ]:
iot_unseen = unseen_data_v2['u_data']
iot_unseen_tgt = unseen_data_v2['u_target']

In [ ]:
from sklearn.model_selection import train_test_split
# Split data into Train/test 25% datasets
(iot_train, iot_test,
 iot_train_tgt, iot_test_tgt) = train_test_split(data_v2['data'], 
                                                 data_v2['target'], 
                                                 test_size=0.25)

In [ ]:
from sklearn import linear_model
from sklearn import discriminant_analysis
from sklearn import naive_bayes
from sklearn import svm
from sklearn import tree
from sklearn import neighbors
import sklearn.metrics as metrics
import sklearn.model_selection as skms
import sklearn.preprocessing as skpre
import sklearn.multiclass as skmulti

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier #
from sklearn.ensemble import AdaBoostClassifier #
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC #
from sklearn.neighbors import KNeighborsClassifier #
from sklearn.tree import DecisionTreeClassifier #

from xgboost import XGBClassifier #

In [ ]:
classifiers = { #
         #'XGB': XGBClassifier(random_state=3, learning_rate=0.05, max_depth=10, n_estimators=100), 
         #'AdaBoost Classifier': AdaBoostClassifier(n_estimators=20)
         #'Random Forest':  RandomForestClassifier()
         #'QDA': discriminant_analysis.QuadraticDiscriminantAnalysis(),
         #'LDA': discriminant_analysis.LinearDiscriminantAnalysis(),
         #'GNB': naive_bayes.GaussianNB()
         'SVC(1)': svm.SVC(kernel="linear"),
         #'SVC(2)': svm.LinearSVC(),#
         #'DTC': tree.DecisionTreeClassifier(),
         #'5NN-C': neighbors.KNeighborsClassifier(),
         #'10NN-C': neighbors.KNeighborsClassifier(n_neighbors=10)
}

In [ ]:
import joblib
pr_unseen=[]
pr_test=[]
for name, model in classifiers.items():
    model.fit(iot_train, iot_train_tgt)
    # Save the trained model to a file
    #joblib.dump(model, 'botnet_model.pkl')
    preds = model.predict(iot_test)
    pr_test.append(preds)
    unseen_preds = model.predict(iot_unseen)
    pr_unseen.append(unseen_preds)



In [ ]:
from sklearn import metrics
from sklearn.model_selection import cross_val_score
Name= 'rawDataRes'
res=[]
output=[]
output2=[]
for (i, v),Xpred in zip(classifiers.items(),pr_test):
    row= {}
    Xpred =  v.predict(iot_test)
    scores = cross_val_score(v, iot_train, iot_train_tgt, cv=5)
    accuracy = metrics.accuracy_score(iot_test_tgt, Xpred)
    confusion_matrix = metrics.confusion_matrix(iot_test_tgt, Xpred)
    classification = metrics.classification_report(iot_test_tgt, Xpred)
  
    lines = classification.split('\n')
    line = lines[7:-1][0]
    row_data = line.split('      ')
    row ['Tst Accuracy'] = accuracy
    row ['Tst CV mean'] = scores.mean()
    row ['Tst Precision'] = float(row_data[1])
    row ['Tst Recall'] = float(row_data[2])
    row ['Tst F1-Score'] = float(row_data[3])
    row ['Tst TN'] = confusion_matrix[0][0]
    row ['Tst FP'] = confusion_matrix[0][1]
    row ['Tst FN'] = confusion_matrix[1][0]
    row ['Tst TP'] = confusion_matrix[1][1]
    output2.append('.......................{} test Model Evaluation =============================='.format(i))
    output2.append("Cross Validation Mean Score: ")
    output2.append(scores.mean())
    output2.append("Model Accuracy: ")
    output2.append(accuracy)
    output2.append("Confusion matrix:")
    output2.append(confusion_matrix)
    output2.append("Classification report:")
    output2.append(classification)
    res.append(row)
#pd.DataFrame.from_dict(res).to_csv('TestResult.csv',index=False)

In [ ]:
#pd.DataFrame.from_dict(res).to_csv('TestResult2.csv',index=False)

In [ ]:
output2

In [ ]:
pd.DataFrame(res)

In [ ]:
# Choose the model you want to visualize
selected_model = list(classifiers.keys())[0]

# Create a single subplot
fig, ax = plt.subplots(figsize=(10, 8))

# Get the selected model and plot its confusion matrix
selected_preds = (classifiers[selected_model].fit(iot_train, iot_train_tgt)
                  .predict(iot_test))
cm = metrics.confusion_matrix(iot_test_tgt, selected_preds)
sns.heatmap(cm, annot=True, cbar=False, ax=ax, fmt="d")
ax.set_title(selected_model)
ax.set_ylabel('Actual')
ax.set_xlabel('Predicted')

plt.show()

In [ ]:
Name= 'rawDataRes2'
res2=[]
output=[]
for (i, v),Xpred_unseen in zip(classifiers.items(),pr_unseen):
    row2= {}
    Xpred_unseen =  v.predict(iot_unseen)
    accuracy_u = metrics.accuracy_score(iot_unseen_tgt, Xpred_unseen)
    confusion_matrix_u = metrics.confusion_matrix(iot_unseen_tgt, Xpred_unseen)
    classification_u = metrics.classification_report(iot_unseen_tgt, Xpred_unseen)
  
    lines2 = classification_u.split('\n')
    line2 = lines2[7:-1][0]
    row_data2 = line2.split('      ')
    row2 ['Unseen Accuracy'] = accuracy_u
    row2 ['Tst Precision'] = float(row_data2[1])
    row2 ['Tst Recall'] = float(row_data2[2])
    row2 ['Tst F1-Score'] = float(row_data2[3])
    row2 ['Tst TN'] = confusion_matrix_u[0][0]
    row2 ['Tst FP'] = confusion_matrix_u[0][1]
    row2 ['Tst FN'] = confusion_matrix_u[1][0]
    row2 ['Tst TP'] = confusion_matrix_u[1][1]
    output.append('.......................{} Unseen Model Evaluation =============================='.format(i))
    output.append("Unseen Model Accuracy: ")
    output.append(accuracy_u)
    output.append("Unseen Confusion matrix:")
    output.append(confusion_matrix_u)
    output.append("Unseen Classification report:")
    output.append(classification_u)
    res2.append(row2)
#pd.DataFrame.from_dict(res2).to_csv('TestResult2.csv',index=False)

In [ ]:
output

In [ ]:
pd.DataFrame(res)

In [ ]:
def compute_readability(text):Text(0.5, 114.00000000000011, 'Predicted')

In [ ]:
# Choose the model you want to visualize
selected_model = list(classifiers.keys())[0]

# Create a single subplot
fig, ax = plt.subplots(figsize=(10, 8))

# Get the selected model and plot its confusion matrix
selected_preds = (classifiers[selected_model].fit(iot_train, iot_train_tgt)
                  .predict(iot_unseen))
cm = metrics.confusion_matrix(iot_unseen_tgt, selected_preds)
sns.heatmap(cm, annot=True, cbar=False, ax=ax, fmt="d")
ax.set_title(selected_model)
ax.set_ylabel('Actual')
ax.set_xlabel('Predicted')

plt.show()